In [54]:
import os 
import openai
import pandas as pd
OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = OPENAI_API_KEY
model1 = 'gpt-3.5-turbo-16k'
model2 = 'gpt-4-0613'

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [55]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.llms.loading import load_llm
from langchain.callbacks import get_openai_callback
from langchain.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory

from langchain.chains import LLMChain


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.7) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [145]:
# Micro and Macro prompts

# Load the data
#input file
question_file = 'Data/Attribution/attribution_by_gics_Obj_1.csv'
answer_file = 'Data/Attribution/attribution_by_gics_Obj2_1.csv'



# Function to generate the prompt
#def generate_prompt_micro(fund, period):

generate_prompt_micro_2 = """
Calculate performance attribution using the Brinson Model for a specific Fund and Period with the provided data. Follow these formulas:
- Allocation Effect = (Portfolio Weight - Benchmark Weight) * (Benchmark Return - Benchmark Total Return)
- Selection Effect = Portfolio Weight * (Portfolio Return - Benchmark Return)
- Total Contribution = Allocation Effect + Selection Effect
- Benchmark Total Return = Weighted average of Benchmark weights and Benchmark returns

This is a multi-level problem where sectors below to 'GICS Type'. 

Objective:
1. Calculate 'Allocation Effect', 'Selection Effect', and 'Total Contribution' for all 'GICS Sector'. Output in CSV format, with the following columns: 
'GICS Sector', 'Allocation Effect', 'Selection Effect', 'Total Contribution', 'Fund', 'Period'. Each record should be on a new line.
2. Calculate 'Allocation Effect', 'Selection Effect', and 'Total Contribution' for all 'GICS Type'. Output in CSV format, with the following columns: 
'GICS Type', 'Allocation Effect', 'Selection Effect', 'Total Contribution', 'Fund', 'Period'. Each record should be on a new line.
3. Output should contain only the CSV format, initiated by 'CSV Format:'. No additional sentences, separators, wordings.

Attention: Use pandas for calculations. Ensure accuracy and adherence to the formulas. 
Attention: Ensure the output contains only the CSV formatted data.

"""


generate_prompt_macro = """
Your objective is to calculate performance attribution based on the Brinson Model for a specific Fund and Period with the provided data. Follow these formulas:
'Allocation Effect' = (Portfolio Weight - Benchmark Weight) * (Benchmark Return - Benchmark Total Return)
'Selection Effect' = Portfolio Weight * (Portfolio Return - Benchmark Return)
'Total Contribution' = 'Allocation Effect' + 'Selection Effect'
'Portfolio Contribution to Return' = Portfolio Weight * Portfolio Return
'Benchmark Contribution to Return' = Benchmark Weight * Benchmark Return
'Benchmark Total Return' is calculated as the weighted average between Benchmark weights and Benchmark returns.
'Portfolio Return' is the sum over 'Portfolio Contribution to Return' divided by the specific level/segment total weight
'Benchmark Return' is the sum over 'Benchmark Contribution to Return' divided by the specific level/segment total weight

This is a multi-level problem where sectors belong to 'GICS Type'. Think carefully on how to aggregate before giving your answers.
Your goal is to calculate Allocation and Selection effect for all 'GICS Type' from the top-down or highest segment, i.e at the 'GICS Type' level, following these steps:
Step 1: Calculate 'Benchmark Total Return'
Step 2: Calculate the 'Portfolio Return' and 'Benchmark Return' at each 'GICS Type' level.
Step 3: Use these results to calculate the 'Allocation Effect', 'Selection Effect' at each 'GICS Type' level.
Attention: Your output is in a CSV format with the following columns: GICS Type, Effect Type, Value, Fund, Period. Each record should be on a new line.
Attention: 'Value' corresponds to the numerical values calculated in Step 3.
Attention: Output should contain the CSV format, initiated by 'CSV Format:', but no additional sentences.
Attention: Use pandas
Attention: Be intentional, You are expected to provide numerical answers and perform calculations following the formulas and the steps provided.
"""



In [146]:
#procedure for MACRO prompt

import time
import ast
from datetime import datetime
from io import StringIO

# Step 1: Read the input CSV file
df = pd.read_csv('attribution_by_gics_Obj2_1.csv')


# Function to save a CSV format DataFrame to a file
def save_csv_temp(fund, period, data):
    output_file = f"temp_{fund}_{period.replace('/', '_')}.csv"
    with open(output_file, 'w') as f:
        f.write(data)
    #print(f"Results saved to {output_file}")
    return output_file

#step 2: save to csv function 
def save_results_to_csv_macro(fund, period, csv_data):
    try:
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
        # Converting the CSV string to a DataFrame

        df = pd.read_csv(StringIO(csv_data))
        
        # Adding Fund and Period columns
        df['Fund'] = fund
        df['Period'] = period
        
        # Reordering columns
        df = df[['GICS Type', 'Effect Type', 'Effect Type Value', 'Fund', 'Period']]
        
        # Saving the DataFrame to a CSV file
        output_file = f"results_{fund}_{period.replace('/', '_')}_{timestamp}.csv"
        df.to_csv(output_file, index=False)
        print(f"Results saved to {output_file}")
        
    except Exception as e:
        print(f"Failed to save results to CSV for Fund: {fund}, Period: {period}. Error: {str(e)}")
   
    
# Step 3: Group by 'Fund' and 'Period'
for (fund, period), group in df.groupby(['Fund', 'Period']):
    # Step 3: Convert the group to a CSV format string 
    csv_data = group.to_csv(index=False)
    temp_file = save_csv_temp(fund, period, csv_data)
    
    #Step 4: generate prompt
    prompt = generate_prompt_macro

    #Step 5: create and call agent
    #agent setup
    memory_chat = ConversationBufferMemory(memory_key="memory_chat", return_messages=True)
    agent = create_csv_agent(
        ChatOpenAI(temperature=0, model=model2),
        temp_file,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        memory=memory_chat,
    )
    
    # Call the Agent
    
    response = agent.run(prompt)
    memory_chat.clear()
    # Format the current time as a string, e.g., "2023-10-23_15-30"
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    
    #step 6: save to csv
    #used for macro
    try:
        # Extracting the CSV-like part from the response text
        csv_data = response.split("CSV Format:\n")[-1].strip()
        #csv_data = response.split("GICS Type,Effect Type,Value\n")[-1].strip()
        print(f"Results for Fund: {fund}, Period: {period}")
        
        # Save results to CSV
        save_results_to_csv_macro(fund, period, csv_data)
    except Exception as e:
        print(f"Failed to parse and save results for Fund: {fund}, Period: {period}. Error: {str(e)}")

  




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\n# Calculate Benchmark Total Return\nbenchmark_total_return = (df['Benchmark Weight'] * df['Benchmark Return']).sum()\n\n# Calculate Portfolio Return and Benchmark Return at each GICS Type level\nportfolio_return = df.groupby('GICS Type')['Portfolio Weight', 'Portfolio Return'].apply(lambda x: (x['Portfolio Weight'] * x['Portfolio Return']).sum() / x['Portfolio Weight'].sum())\nbenchmark_return = df.groupby('GICS Type')['Benchmark Weight', 'Benchmark Return'].apply(lambda x: (x['Benchmark Weight'] * x['Benchmark Return']).sum() / x['Benchmark Weight'].sum())\n\n# Calculate Allocation Effect and Selection Effect at each GICS Type level\nallocation_effect = df.groupby('GICS Type').apply(lambda x: ((x['Portfolio Weight'] - x['Benchmark Weight']) * (benchmark_return - benchmark_total_return)).sum())\nselection_effect = df.groupby('GICS Type').apply(lambda x: (x['Portfolio Weight'] * 

KeyboardInterrupt: 

In [148]:
# procedure to evaluate macro prompt

import pandas as pd
import numpy as np
import os

# List of file names to loop through

file_names=['attribution_by_gics_Obj2_1.csv']

# Replace with the directory where your CSV files are stored
directory_path = './Results_BdMelo/'

# Function to calculate weighted average
def weighted_average(group, weight_col, return_col):
    return (group[weight_col] * group[return_col]).sum() / group[weight_col].sum()

# Loop through each CSV file in the directory
for filename in file_names:
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        
        # Step 1: Read the input CSV file
        df = pd.read_csv(file_path)
        
        # Initialize a DataFrame to store results
        results = []

        # Step 2: Group by 'Fund', 'Period', and 'GICS Type'
        grouped = df.groupby(['Fund', 'Period', 'GICS Type'])
        for (fund, period, gics_type), group in grouped:
            # Calculate weighted averages for the group
            portfolio_weighted_avg = weighted_average(group, 'Portfolio Weight', 'Portfolio Return')
            benchmark_weighted_avg = weighted_average(group, 'Benchmark Weight', 'Benchmark Return')

            # Calculate the 'Benchmark Total Return' for the (Fund, Period) combination
            benchmark_total_return = weighted_average(df[(df['Fund'] == fund) & (df['Period'] == period)], 'Benchmark Weight', 'Benchmark Return')

            # Calculate 'Allocation Effect' and 'Selection Effect'
            allocation_effect = (group['Portfolio Weight'].sum() - group['Benchmark Weight'].sum()) * (benchmark_weighted_avg - benchmark_total_return)
            selection_effect = group['Portfolio Weight'].sum() * (portfolio_weighted_avg - benchmark_weighted_avg)

            # Store the results for this GICS Type
            results.append({
                'Fund': fund,
                'Period': period.replace('/', '_'),
                'GICS Type': gics_type,
                #'Portfolio Weighted Average': portfolio_weighted_avg,
                #'Benchmark Weighted Average': benchmark_weighted_avg,
                #'Benchmark Total Return': benchmark_total_return,
                'Allocation Effect': allocation_effect,
                'Selection Effect': selection_effect
            })

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)
        
        # Optionally, save the results to a CSV file
        output_file_path = os.path.join(directory_path, f'macro_results.csv')
        results_df.to_csv(output_file_path, index=False)

        print(f'Processed {filename} and saved the results to {output_file_path}')

#scoring procedure

import pandas as pd

# Load the CSV files into DataFrames
file_ground_truth = './Results_BdMelo/macro_results.csv'

# Read the ground truth file
df_ground_truth = pd.read_csv(file_ground_truth)
df_ground_truth['Period'] = df_ground_truth['Period'].str.replace('/', '_')

# Define a similarity threshold for comparing the effects
similarity_threshold = 1e-5  # This value can be adjusted as per requirements

# Replace with the directory where your comparison CSV files are stored
directory_path = './Results_BdMelo/'

# List of file names to compare
# Example: 
#'results_Portfolio Growth_7_1_2022 to 9_30_2022_2023-10-29_14-32.csv',
file_names = [
'results_Portfolio Defensive_10_1_2022 to 12_31_2022_2023-10-29_13-47.csv',
'results_Portfolio Defensive_1_31_2022 to 3_31_2022_2023-11-22_20-51.csv',
'results_Portfolio Defensive_4_1_2022 to 6_30_2022_2023-10-29_14-04.csv',
'results_Portfolio Defensive_7_1_2022 to 9_30_2022_2023-10-29_14-06.csv',
'results_Portfolio Growth_1_31_2022 to 3_31_2022_2023-10-29_14-07.csv',
'results_Portfolio Growth_4_1_2022 to 6_30_2022_2023-11-23_12-00.csv',
'results_Portfolio Growth_7_1_2022 to 9_30_2022_2023-10-29_14-32.csv',
'results_Portfolio Growth_10_1_2022 to 12_31_2022_2023-11-22_20-24.csv',
'results_Portfolio Value_1_31_2022 to 3_31_2022_2023-11-23_12-00.csv',
'results_Portfolio Value_7_1_2022 to 9_30_2022_2023-10-30_22-24.csv',
'results_Portfolio _VV_4_1_2022 to 6_30_2022_2023-10-30_22-03.csv',
'results_Portfolio _VV_10_1_2022 to 12_31_2022_2023-11-23_12-00.csv'
]

comparison_results_list = []

# Loop through each file name for comparison
for file_name in file_names:
    if file_name.endswith('.csv') and file_name != 'macro_results.csv':
        # Read the comparison file
        df_to_compare = pd.read_csv(os.path.join(directory_path, file_name))
        df_to_compare['Period'] = df_to_compare['Period'].str.replace('/', '_')
        print(f"Processing file: {file_name}")
        # Pivot and rename columns to match df_ground_truth
        df_to_compare_pivot = df_to_compare.pivot_table(
            index=['Fund', 'Period', 'GICS Type'],
            columns='Effect Type',
            values='Value',
            aggfunc='first'
        ).reset_index()
        #df_to_compare_pivot.columns.name = None  # Remove the categories name
        df_to_compare_pivot.rename(columns={
            'Allocation Effect': 'Allocation Effect_compare',
            'Selection Effect': 'Selection Effect_compare'
        }, inplace=True)

        
        #print(f"Pivoted DataFrame has {df_to_compare_pivot.shape[0]} rows.")

        # Perform the comparison
        comparison_df = pd.merge(
            df_ground_truth,
            df_to_compare_pivot,
            on=['Fund', 'Period', 'GICS Type'],
            how='inner',
            suffixes=('_ground', '_compare')
        )

        # Calculate differences and scores
        comparison_df['Allocation Effect Difference'] = comparison_df['Allocation Effect'] - comparison_df['Allocation Effect_compare']
        comparison_df['Selection Effect Difference'] = comparison_df['Selection Effect'] - comparison_df['Selection Effect_compare']
        comparison_df['Allocation Effect Score'] = (abs(comparison_df['Allocation Effect Difference']) <= similarity_threshold).astype(int)
        comparison_df['Selection Effect Score'] = (abs(comparison_df['Selection Effect Difference']) <= similarity_threshold).astype(int)

        # Add the file name as a column for reference
        comparison_df['Compared File'] = file_name
        
        # Append to the list of results
        comparison_results_list.append(comparison_df)

# Concatenate all results into one DataFrame
all_comparison_results = pd.concat(comparison_results_list)

# Save the final results to a new CSV file
output_file = './Results_BdMelo/all_comparison_results_11_23.csv'
all_comparison_results.to_csv(output_file, index=False)

print(f"All comparison results saved to {output_file}.")

Processed attribution_by_gics_Obj2_1.csv and saved the results to ./Results_BdMelo/macro_results.csv
Processing file: results_Portfolio Defensive_10_1_2022 to 12_31_2022_2023-10-29_13-47.csv
Processing file: results_Portfolio Defensive_1_31_2022 to 3_31_2022_2023-11-22_20-51.csv
Processing file: results_Portfolio Defensive_4_1_2022 to 6_30_2022_2023-10-29_14-04.csv
Processing file: results_Portfolio Defensive_7_1_2022 to 9_30_2022_2023-10-29_14-06.csv
Processing file: results_Portfolio Growth_1_31_2022 to 3_31_2022_2023-10-29_14-07.csv
Processing file: results_Portfolio Growth_4_1_2022 to 6_30_2022_2023-11-23_12-00.csv
Processing file: results_Portfolio Growth_7_1_2022 to 9_30_2022_2023-10-29_14-32.csv
Processing file: results_Portfolio Growth_10_1_2022 to 12_31_2022_2023-11-22_20-24.csv
Processing file: results_Portfolio Value_1_31_2022 to 3_31_2022_2023-11-23_12-00.csv
Processing file: results_Portfolio Value_7_1_2022 to 9_30_2022_2023-10-30_22-24.csv
Processing file: results_Portfol

In [132]:
# procedure to evaluate micro prompt

import pandas as pd
import numpy as np
import os


# Function to ensure columns are numeric
def to_numeric(column):
    return pd.to_numeric(column, errors='coerce')

# Define the directory path and file names
directory_path = './Results_BdMelo/'

# Define a tolerance for the score calculation
tolerance = 1e-5  

# Read the ground truth data
df_attribution = pd.read_csv('attribution_by_gics_Obj_1.csv')

# Ensure the 'Allocation Effect' column in df_attribution is numeric
df_attribution['Allocation Effect'] = to_numeric(df_attribution['Allocation Effect'])
df_attribution['Selection Effect'] = to_numeric(df_attribution['Selection Effect'])
df_attribution['Total Contribution'] = to_numeric(df_attribution['Total Contribution'])

# Define the GICS mapping
# Create the mapping DataFrame
gics_mapping = pd.DataFrame({
    'GICS Sector': [
        'Consumer Discret.', 'Financials', 'Materials', 'Real Estate', 
        'Consumer Staples', 'Health Care', 'Utilities', 'Cash',
        'Communication', 'Energy', 'Industrials', 'IT'
    ],
    'GICS Type': [
        'Cyclical', 'Cyclical', 'Cyclical', 'Cyclical',
        'Defensive', 'Defensive', 'Defensive', 'Defensive',
        'Sensitive', 'Sensitive', 'Sensitive', 'Sensitive'
    ]
})

# Merge the mapping with df_attribution and aggregate by GICS Type
df_attribution_mapped = df_attribution.merge(gics_mapping, on='GICS Sector')
df_attribution_by_type = df_attribution_mapped.groupby(['GICS Type','Fund','Period']).sum().reset_index()

# Function to calculate the score based on tolerance
def calculate_score(value1, value2):
    return int(abs(value1 - value2) <= tolerance)

# Initialize a list to store scores for each file
comparison_scores = []

# Loop through each CSV file in the directory and process them
for file_name in os.listdir(directory_path):
    if file_name.startswith('Micro_results') and file_name.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory_path, file_name))
        
        print(f'Processing {file_name}')

        # Split the DataFrame into parts
        # df_part1 contains the first 13 rows (1 header + 12 data rows)
        df_part1 = df.iloc[:12]

        # df_part2 starts from row 13, which contains the new header
        # First, extract the data starting from the row 13 to the end
        df_part2_data = df.iloc[12:]

        # Now use the first row of this new DataFrame as the header for df_part2
        new_header = df_part2_data.iloc[0]  # capture the header
        df_part2 = df_part2_data[1:]  # take the data less the header row
        df_part2.columns = new_header  # set the header row as the df header

        # Reset index for both DataFrames if needed
        df_part1.reset_index(drop=True, inplace=True)
        df_part2.reset_index(drop=True, inplace=True)
        
        # Ensure the 'Allocation Effect' column in df_attribution is numeric
        #df_part1['Allocation Effect'] = to_numeric(df_part1['Allocation Effect'])
        #df_part1['Selection Effect'] = to_numeric(df_part1['Selection Effect'])
        #df_part1['Total Contribution'] = to_numeric(df_part1['Total Contribution'])
        df_part1.loc[:, 'Allocation Effect'] = pd.to_numeric(df_part1.loc[:, 'Allocation Effect'], errors='coerce')
        df_part1.loc[:, 'Selection Effect'] = pd.to_numeric(df_part1.loc[:, 'Selection Effect'], errors='coerce')
        df_part1.loc[:, 'Total Contribution'] = pd.to_numeric(df_part1.loc[:, 'Total Contribution'], errors='coerce')

       # df_part2['Allocation Effect'] = to_numeric(df_part2['Allocation Effect'])
       # df_part2['Selection Effect'] = to_numeric(df_part2['Selection Effect'])
       # df_part2['Total Contribution'] = to_numeric(df_part2['Total Contribution'])
        # Convert columns to numeric in df_part2, using .loc to avoid SettingWithCopyWarning
        df_part2.loc[:, 'Allocation Effect'] = pd.to_numeric(df_part2.loc[:, 'Allocation Effect'], errors='coerce')
        df_part2.loc[:, 'Selection Effect'] = pd.to_numeric(df_part2.loc[:, 'Selection Effect'], errors='coerce')
        df_part2.loc[:, 'Total Contribution'] = pd.to_numeric(df_part2.loc[:, 'Total Contribution'], errors='coerce')

        # Perform the comparison for part1
        # Merge with the attribution data
        merged_df = df_attribution.merge(df_part1, on=['Fund', 'Period', 'GICS Sector'])
        
        # Calculate scores
        merged_df['Allocation Score'] = merged_df.apply(lambda row: calculate_score(row['Allocation Effect_x'], row['Allocation Effect_y']), axis=1)
        merged_df['Selection Score'] = merged_df.apply(lambda row: calculate_score(row['Selection Effect_x'], row['Selection Effect_y']), axis=1)
        merged_df['Total Contr Score'] = merged_df.apply(lambda row: calculate_score(row['Total Contribution_x'], row['Total Contribution_y']), axis=1)
        # Compile the scores into the comparison_scores list
        scores = {
            'filename': file_name,
            'Period': merged_df['Period'].iloc[0],
            'Portfolio': merged_df['Fund'].iloc[0],  # Assuming Fund is the same for all rows
            'Part1_Allocation Score': merged_df['Allocation Score'].sum(),
            'Part1_Selection Score': merged_df['Selection Score'].sum(),
            'Part1_Total Contr Score': merged_df['Total Contr Score'].sum()
        }
        comparison_scores.append(scores)        
        
        # Initialize score accumulators for part2
        part2_allocation_scores = []
        part2_selection_scores = []
        part2_total_contr_scores = []
        
        # Iterate over df_part2 to calculate individual scores
        for _, row in df_part2.iterrows():
            # Find the corresponding row in df_attribution_by_type
            attr_row = df_attribution_by_type[(df_attribution_by_type['Fund'] == row['Fund']) & 
                                              (df_attribution_by_type['Period'] == row['Period']) & 
                                              (df_attribution_by_type['GICS Type'] == row['GICS Type'])]

            if not attr_row.empty:
                # Calculate individual scores and add them to the lists
                part2_allocation_scores.append(calculate_score(row['Allocation Effect'], attr_row['Allocation Effect'].iloc[0]))
                part2_selection_scores.append(calculate_score(row['Selection Effect'], attr_row['Selection Effect'].iloc[0]))
                part2_total_contr_scores.append(calculate_score(row['Total Contribution'], attr_row['Total Contribution'].iloc[0]))

        # Sum the scores for all GICS Types
        part2_allocation_score_sum = sum(part2_allocation_scores)
        part2_selection_score_sum = sum(part2_selection_scores)
        part2_total_contr_score_sum = sum(part2_total_contr_scores)
        
        # Find the index in combined_scores to update with part2 scores
        score_index = next((index for (index, d) in enumerate(comparison_scores) if d['filename'] == file_name and d['Portfolio'] == row['Fund'] and d['Period'] == row['Period']), None)
        if score_index is not None:
            # Update the existing dictionary with Part2_ scores
            comparison_scores[score_index].update({
                'Part2_Allocation Score': part2_allocation_score_sum,
                'Part2_Selection Score': part2_allocation_score_sum,
                'Part2_Total Contr Score': part2_allocation_score_sum
            })
# Convert the combined scores list to a DataFrame
final_scores_df = pd.DataFrame(comparison_scores)

# Save the final scores DataFrame to a CSV file
final_output_file = './Results_BdMelo/Micro_final_combined_scores.csv'
final_scores_df.to_csv(final_output_file, index=False)

# Return the path to the final output file
#final_output_file



Processing Micro_results_Portfolio Growth_1_31_2022 to 3_31_2022_2023-10-31_12-25_part_1.csv
Processing Micro_results_Portfolio Defensive_4_1_2022 to 6_30_2022_2023-10-31_12-19_part_1.csv
Processing Micro_results_Portfolio VVV_1_31_2022 to 3_31_2022_2023-10-31_12-36_part_1.csv
Processing Micro_results_Portfolio Defensive_10_1_2022 to 12_31_2022_2023-10-31_12-17_part_1.csv
Processing Micro_results_Portfolio Defensive_7_1_2022 to 9_30_2022_2023-10-31_12-22_part_1.csv
Processing Micro_results_Portfolio Growth_7_1_2022 to 9_30_2022_2023-10-31_12-33_part_1.csv
Processing Micro_results_Portfolio VVV_4_1_2022 to 6_30_2022_2023-10-31_12-42_part_1.csv
Processing Micro_results_Portfolio Defensive_1_31_2022 to 3_31_2022_2023-10-31_12-14_part_1.csv
Processing Micro_results_Portfolio VVV_7_1_2022 to 9_30_2022_2023-10-31_12-44_part_1.csv
Processing Micro_results_Portfolio Growth_4_1_2022 to 6_30_2022_2023-10-31_12-31_part_1.csv
Processing Micro_results_Portfolio VVV_10_1_2022 to 12_31_2022_2023-10-3

In [7]:
#Questions pipeline

import pandas as pd
import random

#input file
question_file = 'attribution_by_gics_Obj_1.csv'
answer_file = 'attribution_by_gics_Obj2_1.csv'


# Sample data
data = pd.read_csv(question_file)

df = pd.DataFrame(data)

# one type
def generate_random_questions_1(df, num_questions=10, precision=5):
    questions = []
    #short_questions = []
    columns = ["Allocation Effect", "Selection Effect", "Total Contribution"]
    
    for _ in range(num_questions):
        # Randomly select a sector, a column, a fund, and a period
        row = df.sample().iloc[0]
        column_name = random.choice(columns)
        fund = row['Fund']
        period = row['Period']
        #fund = random.choice(df['Fund'].unique())
        #period = random.choice(df['Period'].unique())
        
        # Generate the question based on the selected sector and column
        actual_value = row[column_name]
        rounded_value = round(actual_value, precision)
        
        # Generate distractors from other rows in the DataFrame
        distractors = set()
        while len(distractors) < 3:
            distractor_row = df.sample().iloc[0]
            if distractor_row.name != row.name:
                distractor_value = round(distractor_row[column_name], precision)
                if distractor_value != rounded_value:
                    distractors.add(distractor_value)
        
        options = [rounded_value] + list(distractors)
        random.shuffle(options)
        
        question = {
            "question": f"The {column_name.replace('_', ' ').title()} from the {row['GICS Sector']} sector, in fund {fund}, in the period {period}, is:",
            "options": options,
            "correct_answer": rounded_value
            }
        questions.append(question)
            
        #short_question = {            
        #    "question": f"Calculate the {column_name.replace('_', ' ').title()} from the {row['GICS Sector']} sector, in fund {fund}, in the period {period}:",
        #    "correct_answer": rounded_value    
        #}
        #short_questions.append(short_question)
    
    return questions

import random

#two types
def generate_random_questions(df, num_questions=10, precision=5):
    questions = []
    columns = ["Allocation Effect", "Selection Effect", "Total Contribution"]
    
    for i in range(num_questions):
        # Randomly select a sector, a column, and a question type
        row = df.sample().iloc[0]
        column_name = random.choice(columns)
        fund = row['Fund']
        period = row['Period']
        
        # Determine the question type based on the iteration
        if i < num_questions // 2:
            question_type = "value"
            question_text = f"The {column_name.replace('_', ' ').title()} from the {row['GICS Sector']} sector, in fund {fund}, in the period {period}, is closest to:"
        else:
            question_type = "calculation"
            question_text = f"Calculate the {column_name.replace('_', ' ').title()} from the {row['GICS Sector']} sector, in fund {fund}, in the period {period}:"
        
        # Generate the question based on the selected sector, column, and question type
        actual_value = row[column_name]
        rounded_value = round(actual_value, precision)
        
        # Generate distractors from other rows in the DataFrame
        distractors = set()
        while len(distractors) < 3:
            distractor_row = df.sample().iloc[0]
            if distractor_row.name != row.name and distractor_row['Fund'] == fund and distractor_row['Period'] == period:
                distractor_value = round(distractor_row[column_name], precision)
                if distractor_value != rounded_value:
                    distractors.add(distractor_value)
        
        options = [rounded_value] + list(distractors)
        random.shuffle(options)
        
        question = {
            "question": question_text,
            "options": options,
            "correct_answer": rounded_value,
            "question_type": question_type
        }
        questions.append(question)
        
    return questions



In [8]:
# formating question into the prompt
# creating agent
# running llm

from langchain.callbacks import get_openai_callback

#one type
def format_question_1(question):
    choices = "\n".join([f"{letter}: {option}%" for letter, option in zip("ABCD", question['options'])])
    prompt = f"""
    Question below is about performance attribution, use instructions to find a solution and follow these guidelines:
    Guidelines:
    -You are giving four choices to answer the question. You must choose one of the answers and your output is just the first letter of that answer, nothing else. 
    -Answer 'E' if you do not know the answer.
    -Before giving final answer, recall relevant problems and solutions.
    Instructions:
    *'Allocation Effect' = (Portfolio Weight - Benchmark Weight) * (Benchmark Return - Benchmark Total Return)
    *'Selection Effect' = Portfolio Weight * (Portfolio Return - Benchmark Return)
    *'Total Contribution' = 'Allocation Effect' + 'Selection Effect'
    *'Benchmark Total Return' is calculated as the weighted average between Benchmark weights and Benchmark returns
    Question:
    Using the Brinson Model, {question['question']}
    Choices:
    {choices}
    Answer:
    """
    return prompt

#two types
#prompt type 1
def format_question_2(question):
    """Formats the question for GPT-4."""
    options_text = "\n".join([f"{letter}: {opt}%" for letter, opt in zip("ABCD", question['options'])])
    
    if "is closest to" in question["question"]:
        # It's a "value" question
        prompt = f"""
        Question below asks you to calculate performance attribution based on the Brinson Model.

        You know that for a specific Fund and Period:
        'Allocation Effect' = (Portfolio Weight - Benchmark Weight) * (Benchmark Return - Benchmark Total Return)
        'Selection Effect' = Portfolio Weight * (Portfolio Return - Benchmark Return)
        'Total Contribution' = 'Allocation Effect' + 'Selection Effect'
        'Benchmark Total Return' is calculated as the weighted average between columns 'Benchmark weight' and 'Benchmark return'
        
        You are giving four choices to answer the question. You must choose one of the answers and your output is just the first letter of that answer, nothing else.
        Answer 'E' if you do not know the answer.
        Attention: Data provided contains various funds and periods. Choose correctly.
        Question:
        Solve step-by-step, first extract the Fund and Period provided in the question, second calculate 'Benchmark Total Return' for all 'GICS Sector' filtered by the Fund and Period extracted, 
        third calculate Allocation and Selection Effect for all 'GICS Sector' filtered by the Fund and Period extracted; and finally answer the following: {question["question"]}
        Choices:
        {options_text}
        Answer:"""
    else:
        # It's a "calculation" question
        prompt = f"""
        Question below asks you to calculate performance attribution based on the Brinson Model. Make sure to output the answer as a numerical value rounded to five decimal places.
        
        You know that for a specific Fund and Period:
        'Allocation Effect' = (Portfolio Weight - Benchmark Weight) * (Benchmark Return - Benchmark Total Return)
        'Selection Effect' = Portfolio Weight * (Portfolio Return - Benchmark Return)
        'Total Contribution' = 'Allocation Effect' + 'Selection Effect'
        'Benchmark Total Return' is calculated as the weighted average between columns 'Benchmark weight' and 'Benchmark return'
        
        Note: Your answer should be a single numerical value rounded to five decimal places.
        Attention: Data provided contains various funds and periods. Choose correctly.
        Question:
        Solve step-by-step, first extract the Fund and Period provided in the question, second calculate 'Benchmark Total Return' for all 'GICS Sector' filtered by the Fund and Period extracted, 
        third calculate Allocation and Selection Effect for all 'GICS Sector' filtered by the Fund and Period extracted; and finally answer the following: {question["question"]}
        Note: Your answer should be a single numerical value rounded to five decimal places, nothing else.
        Answer:"""
    return prompt



#prompt type 2
def format_question_3(question):
    """Formats the question for GPT-4."""
    options_text = "\n".join([f"{letter}: {opt}%" for letter, opt in zip("ABCD", question['options'])])
    
    if "is closest to" in question["question"]:
        # It's a "value" question
        prompt = f"""
        Question below is about performance attribution, use instructions to find a solution based on provided data and follow these guidelines:
        Guidelines:
        -You are giving four choices to answer the question. You must choose one of the answers and your output is just the first letter of that answer, nothing else. 
        -Answer 'E' if you do not know the answer.
        
        Instructions:
        *'Allocation Effect' = (Portfolio Weight - Benchmark Weight) * (Benchmark Return - Benchmark Total Return)
        *'Selection Effect' = Portfolio Weight * (Portfolio Return - Benchmark Return)
        *'Total Contribution' = 'Allocation Effect' + 'Selection Effect'
        *'Benchmark Total Return' is calculated as the weighted average between columns 'Benchmark weight' and 'Benchmark return'
        Question:
        Using the Brinson Model, {question['question']}
        Choices:
        {options_text}
        Answer:"""
    else:
        # It's a "calculation" question
        prompt = f"""
        In the context of performance attribution analysis using the Brinson Model, calculate the numerical value based on the provided information and instructions. Make sure to output the answer as a numerical value rounded to five decimal places.
        Do not include any additional text or characters in your answer.
        Instructions:
        - 'Allocation Effect' is calculated as (Portfolio Weight - Benchmark Weight) * (Benchmark Return - Benchmark Total Return).
        - 'Selection Effect' is calculated as Portfolio Weight * (Portfolio Return - Benchmark Return).
        - 'Total Contribution' is the sum of 'Allocation Effect' and 'Selection Effect'.
        - 'Benchmark Total Return' is calculated as the weighted average between columns 'Benchmark weight' and 'Benchmark return'
        
        Example:
        If the Portfolio Weight is 0.1, Benchmark Weight is 0.4, Portfolio Return is 0.05, Benchmark Return is 0.2, and Benchmark Total Return is 0.126, then:
        - Allocation Effect = (0.1 - 0.4) * (0.2 - 0.126) = -0.0222
        - Selection Effect = 0.1 * (0.05 - 0.2) = -0.015
        - Total Contribution = -0.0222 - 0.015 = -0.0372
        Example for 'Benchmark Total Return':
        Suppose you have the following data:
        Benchmark Weight: [0.2, 0.4, 0.4]
        Benchmark Return: [0.15, 0.2, 0.04]
        Benchmark Total Return = [(0.2 * 0.15) + (0.4 * 0.2) + (0.4 * 0.04)] / (0.2 + 0.4 + 04) = 0.126

        Note: Your answer should be a single numerical value rounded to five decimal places.

        Question: 
        {question["question"]}
        Answer:
        """
    return prompt
#-Before giving final answer, recall relevant problems and solutions.

def ask_gpt_openai(question, temperature):
    formatted_question = format_question_2({"question": question["question"], "options": question["options"]})
    
    #agent setup
    agent = create_csv_agent(
        ChatOpenAI(temperature=temperature, model=model2),
        answer_file,
        verbose=False,
        agent_type=AgentType.OPENAI_FUNCTIONS
    ) 
    memory_chat = ConversationBufferMemory()
    response = agent.run(formatted_question)
    memory_chat.clear()
    return response

def ask_gpt_openai_token(question, temperature):
    formatted_question = format_question_2({"question": question["question"], "options": question["options"]})
    with get_openai_callback() as cb:
        #agent setup
        agent = create_csv_agent(
            ChatOpenAI(temperature=temperature, model=model2),
            answer_file,
            verbose=False,
            agent_type=AgentType.OPENAI_FUNCTIONS
        ) 
        memory_chat = ConversationBufferMemory()
        response = agent.run(formatted_question)
        total_tk = cb.total_tokens
        tk_prompt = cb.prompt_tokens
        tk_completion = cb.completion_tokens
        total_cost = cb.total_cost
        memory_chat.clear()
    return response, total_tk, tk_prompt, tk_completion, total_cost

# Generate 50 random questions
questions = generate_random_questions(df, num_questions=10)




In [9]:
import csv
from datetime import datetime

# Temperatures to loop over
temperatures = [0]

# agent types
#ag_types = ['AgentType.OPENAI_FUNCTIONS','AgentType.ZERO_SHOT_REACT_DESCRIPTION']

# Prepare data for CSV
csv_data = []

#QA

for temperature in temperatures:
    # Ask GPT-4 the questions one by one and compile the answers
    gpt_responses = [ask_gpt_openai(question, temperature) for question in questions]
   
        # Check if the answer is correct
        #is_correct = is_answer_correct(question, gpt_response)

    # Store results in CSV data
    for i, (question, gpt_response) in enumerate(zip(questions, gpt_responses), start=1):
        correct_option = "ABCD"[question['options'].index(question['correct_answer'])]
        correct_value = question['correct_answer']

        question_type = question['question_type']

        question_prompt = question['question']

        if question_type == "value":
            correct_option = "ABCD"[question['options'].index(question['correct_answer'])]
            is_correct = 1 if correct_option == gpt_response else 0
        elif question_type == "calculation":
            try:
                gpt_response_number = round(float(gpt_response.strip()), 5)
            except ValueError:
                gpt_response_number = None
            is_correct = 1 if gpt_response_number == question['correct_answer'] else 0


        csv_data.append({
            "Question Number": i,
            "Question Type": question_type,
            "Question": question_prompt,
            "LLM Answer": gpt_response,
            "Correct Choice": correct_option,
            "Correct Value": correct_value,
            "Is Correct": is_correct,
            "Temperature": temperature,
            "Agent Type": "OpenAI_Functions",
            "Model Type": "GPT-4"

        })

        
# Format the current time as a string, e.g., "2023-10-23_15-30"
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")

   
# Write results to CSV
filename = f'gpt4_results_{timestamp}.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["Question Number", "Question Type", "Question", "LLM Answer", "Correct Choice", "Correct Value", "Is Correct", "Temperature", "Agent Type", "Model Type"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(csv_data)

print(f"Results saved to '{filename}'")




<string>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Results saved to 'gpt4_results_2023-11-22_12-38.csv'


In [14]:
import pandas as pd
import os

# Define the directory where the files are located
directory = "./Results_BdMelo"  # Replace with your folder path

# List of filenames
file_names = [
"gpt4_results_2023-10-28_13-53.csv",
"gpt4_results_2023-10-28_14-06.csv",
"gpt4_results_2023-10-28_17-48.csv",
"gpt4_results_2023-10-28_17-53.csv",
"gpt4_results_2023-10-28_17-57.csv",
"gpt4_results_2023-10-28_18-01.csv",
"gpt4_results_2023-10-28_19-05.csv",
"gpt4_results_2023-11-13_00-19.csv",
"gpt4_results_2023-11-13_00-24.csv",
"gpt4_results_2023-11-13_00-27.csv",
"gpt4_results_2023-11-13_00-33.csv",
"gpt4_results_2023-11-13_00-43.csv",
"gpt4_results_2023-11-22_12-38.csv"

]


# Concatenate all dataframes
all_dfs = []
for file_name in file_names:
    file_path = os.path.join(directory, file_name)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df['FileName'] = file_name
        all_dfs.append(df)
        
combined_df = pd.concat(all_dfs, ignore_index=True)

# Save the combined DataFrame before grouping
combined_df.to_csv("./Results_BdMelo/combined_output.csv", index=False)

# Group by 'Question Type' and aggregate
grouped = combined_df.groupby('Question Type').agg(
    Correct_Sum=('Is Correct', 'sum'),
    Total_Questions=('Is Correct', 'count')
)

# Calculate the ratio of correct answers
grouped['Correct_Ratio'] = grouped['Correct_Sum'] / grouped['Total_Questions']

# Reset index for better readability
grouped_reset = grouped.reset_index()

# Display or save the grouped DataFrame
print(grouped_reset)
grouped_reset.to_csv("./Results_BdMelo/output_grouped_file.csv", index=False)  # Save to CSV



  Question Type  Correct_Sum  Total_Questions  Correct_Ratio
0   calculation           59               70       0.842857
1         value           62               70       0.885714


In [10]:
# token cost implementation
import csv
from datetime import datetime

# Temperatures to loop over
temperatures = [0]

# agent types
#ag_types = ['AgentType.OPENAI_FUNCTIONS','AgentType.ZERO_SHOT_REACT_DESCRIPTION']

# Prepare data for CSV
csv_data = []

#QA

for temperature in temperatures:
    # Ask GPT-4 the questions one by one and compile the answers
    #gpt_responses = [ask_gpt_openai(question, temperature) for question in questions]
    
    # Initialize lists to store the results
    gpt_responses = []
    total_token = []
    token_prompt = []
    token_completion = []
    total_cost_usd = []

    # Ask GPT-4 the questions one by one and compile the answers
    #gpt_responses = [ask_gpt_openai(question, temperature) for question in questions]
    for question in questions:
        response, total_tk, tk_prompt, tk_completion, total_cost = ask_gpt_openai_token(question, temperature)
        gpt_responses.append(response)
        total_token.append(total_tk)
        token_prompt.append(tk_prompt)
        token_completion.append(tk_completion)
        total_cost_usd.append(total_cost)
    
    
        # Check if the answer is correct
        #is_correct = is_answer_correct(question, gpt_response)

    # Store results in CSV data
    for i, (question, gpt_response) in enumerate(zip(questions, gpt_responses), start=1):
        correct_option = "ABCD"[question['options'].index(question['correct_answer'])]
        correct_value = question['correct_answer']

        question_type = question['question_type']

        question_prompt = question['question']

        if question_type == "value":
            correct_option = "ABCD"[question['options'].index(question['correct_answer'])]
            is_correct = 1 if correct_option == gpt_response else 0
        elif question_type == "calculation":
            try:
                gpt_response_number = round(float(gpt_response.strip()), 5)
            except ValueError:
                gpt_response_number = None
            is_correct = 1 if gpt_response_number == question['correct_answer'] else 0


        csv_data.append({
            "Question Number": i,
            "Question Type": question_type,
            "Question": question_prompt,
            "LLM Answer": gpt_response,
            "Correct Choice": correct_option,
            "Correct Value": correct_value,
            "Is Correct": is_correct,
            "Temperature": temperature,
            "Agent Type": "OpenAI_Functions",
            "Model Type": "GPT-4"

        })

    data_cost = {
        "Total Token": total_token,
        "Prompt Tokens": token_prompt,
        "Complete Tokens": token_completion,
        "Total Cost (USD)": total_cost_usd
    }
    df_cost = pd.DataFrame(data_cost)        
        
        
# Format the current time as a string, e.g., "2023-10-23_15-30"
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")

#cost file
filename_cost = f"cost_{timestamp}.csv"
df_cost.to_csv(filename_cost, index=False)
print(f"Cost saved to '{filename}'")
    
    
# Write results to CSV
filename = f'gpt4_results_{timestamp}.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["Question Number", "Question Type", "Question", "LLM Answer", "Correct Choice", "Correct Value", "Is Correct", "Temperature", "Agent Type", "Model Type"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(csv_data)

print(f"Results saved to '{filename}'")




<string>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

KeyboardInterrupt

